## 1. Import Libraries

In [51]:
get_ipython().system(u' pip install --upgrade pip')
get_ipython().system(u' pip install beautifulsoup4')
!pip install lxml
!pip install html5lib
!pip install requests

#Importing packages
import requests
from bs4 import BeautifulSoup

Requirement already up-to-date: pip in /opt/conda/envs/Python36/lib/python3.6/site-packages (20.1.1)


In [2]:

import pandas as pd
import numpy as np
import csv

In [3]:
pd.set_option('max_colwidth', 800)

## 2. Scrap data from Wikipedia page into a DataFrame

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Chennai').text 
soup = BeautifulSoup(source, 'lxml')

In [5]:
csv_file = open('Chennai.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Neighbourhood'])

15

In [6]:
mwcg = soup.find_all(class_ = "mw-category-group")

length = len(mwcg) # Gets the length of number of `mw-category-groups` present

for i in range(1, length):  # Gets all the neighbourhoods
    lists = mwcg [i].find_all('a')
    for list in lists:
        nbd = list.get('title') # Gets the title of the neighbourhood
        csv_writer.writerow([nbd]) # Writes the name of the neighbourhood in the csv file

In [7]:
csv_file.close()

In [8]:
df = pd.read_csv('Chennai.csv')

In [9]:
df.shape

(199, 1)

In [10]:
df.head()

,Neighbourhood
0,Adambakkam
1,"Adyar, Chennai"
2,Agaram
3,Alandur
4,Alapakkam


In [12]:


!pip install folium

     |████████████████████████████████| 93 kB 2.9 MB/s eta 0:00:011


In [13]:
!pip install geocoder

     |████████████████████████████████| 98 kB 5.4 MB/s eta 0:00:011


In [14]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
print("Library Imported")

Library Imported


##  3. To Get the Geographical coordinates

In [15]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    

In [16]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighbourhood"].tolist() ]

In [17]:
coords

[[12.99192000000005, 80.20603000000006],
 [13.00305000000003, 80.25193000000007],
 [13.109530000000063, 80.23236000000009],
 [13.00013000000007, 80.20060000000007],
 [13.046100000000024, 80.16499000000005],
 [13.034710000000075, 80.25416000000007],
 [13.050550000000044, 80.18397000000004],
 [13.071390000000065, 80.22256000000004],
 [13.175620000000038, 80.25418000000008],
 [13.191590000000076, 80.27328000000006],
 [13.083590000000072, 80.21020000000004],
 [13.092720000000043, 80.20222000000007],
 [13.022580000000062, 80.22556000000003],
 [13.066770000000076, 80.19419000000005],
 [13.073080000000061, 80.20952000000005],
 [13.035390000000064, 80.21220000000005],
 [13.164570000000026, 80.23274000000004],
 [13.256700000000023, 80.29109000000005],
 [13.248790000000042, 80.31993000000006],
 [13.09883000000002, 80.23238000000003],
 [12.976560000000063, 80.18642000000006],
 [13.083620000000053, 80.28252000000003],
 [13.064400000000035, 80.28065000000004],
 [13.072880000000055, 80.2456700000000

In [18]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_Chennaicoords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [19]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_Chennaicoords['Latitude']
df['Longitude'] = df_Chennaicoords['Longitude']

In [20]:
df.shape

(199, 3)

In [21]:
df_chennai = df
df_chennai.head()

,Neighbourhood,Latitude,Longitude
0,Adambakkam,12.99192,80.20603
1,"Adyar, Chennai",13.00305,80.25193
2,Agaram,13.10953,80.23236
3,Alandur,13.00013,80.20060
4,Alapakkam,13.04610,80.16499


In [22]:
# save the DataFrame as CSV file
df_chennai.to_csv("Chennai.csv", index=False)

## 4. Create a map of Chennai with neighborhoods

In [23]:
address = 'Chennai, Tamil Nadu'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, TN {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai, TN 13.0801721, 80.2838331.


In [24]:
# create map of Toronto using latitude and longitude values
map_Chennai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_chennai['Latitude'], df_chennai['Longitude'], df_chennai['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Chennai)  
    
map_Chennai

In [25]:
# save the map as HTML file
map_Chennai.save('ChennaiMap.html')

## 5. Use the Foursquare API to explore the neighborhoods

In [26]:
# define Foursquare Credentials and Version
CLIENT_ID = 'QEZTYURJMBAZQRAITSQYXJQBHTY0P13EOX311OVSGRA45ZTW' # your Foursquare ID
CLIENT_SECRET = 'QBDRWFWLS4OOVVBTXWUT12TQ3DHVIUTECQ5NPYNVZQHP44QF' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QEZTYURJMBAZQRAITSQYXJQBHTY0P13EOX311OVSGRA45ZTW
CLIENT_SECRET:QBDRWFWLS4OOVVBTXWUT12TQ3DHVIUTECQ5NPYNVZQHP44QF


In [27]:
radius=500
LIMIT=100
venues = []
for lat, long, neighborhood in zip(df_chennai['Latitude'], df_chennai['Longitude'], df_chennai['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [28]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(932, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Adyar, Chennai",13.00305,80.25193,"Nalli, Adayar",13.002910,80.251950,Women's Store
1,"Adyar, Chennai",13.00305,80.25193,That Madras Place,13.005848,80.250726,Café
2,"Adyar, Chennai",13.00305,80.25193,ibaco,13.005864,80.251764,Ice Cream Shop
3,"Adyar, Chennai",13.00305,80.25193,Wonton,13.005047,80.251690,Chinese Restaurant
4,"Adyar, Chennai",13.00305,80.25193,Domino's Pizza,13.002000,80.254000,Pizza Place


In [29]:
# One hot encoding
Chennai_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
Chennai_onehot['Neighbourhood'] = venues_df['Neighborhood'] 

# Move neighborhood column to the first column
fixed_columns = [Chennai_onehot.columns[-1]] + Chennai_onehot.columns[:-1].values.tolist()
Chennai_onehot = Chennai_onehot[fixed_columns]

Chennai_onehot.head()

,Neighbourhood,ATM,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Tea Room,Tennis Court,Thai Restaurant,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Water Park,Women's Store,Yoga Studio
0,"Adyar, Chennai",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,"Adyar, Chennai",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Adyar, Chennai",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Adyar, Chennai",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Adyar, Chennai",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 144 uniques categories.


In [31]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(["Women's Store", 'Café', 'Ice Cream Shop', 'Chinese Restaurant',
       'Pizza Place', 'Indian Restaurant',
       'Vegetarian / Vegan Restaurant', 'Bookstore',
       'North Indian Restaurant', 'Fast Food Restaurant',
       'Italian Restaurant', 'Department Store', 'Movie Theater',
       'Asian Restaurant', 'Gourmet Shop', 'Coffee Shop', 'Gym',
       'Clothing Store', 'Chettinad Restaurant', 'Breakfast Spot',
       'Train Station', 'Burger Joint', 'Metro Station', 'IT Services',
       'Sandwich Place', 'Restaurant', 'Bakery', 'Snack Place',
       'Tea Room', 'Thai Restaurant', 'Japanese Restaurant', 'Juice Bar',
       "Men's Store", 'Bistro', 'Furniture / Home Store',
       'Organic Grocery', 'Food & Drink Shop', 'Multiplex',
       'Shopping Mall', 'Electronics Store', 'Fruit & Vegetable Store',
       'Pharmacy', 'Shoe Store', 'Park', 'South Indian Restaurant',
       'American Restaurant', 'BBQ Joint', 'Farmers Market',
       'Fish Market', 'Bus Station'], dtype=obj

In [32]:
Chennai_grouped = Chennai_onehot.groupby(["Neighbourhood"]).mean().reset_index()

print(Chennai_grouped.shape)
Chennai_grouped.head()

(156, 145)


,Neighbourhood,ATM,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,...,Tea Room,Tennis Court,Thai Restaurant,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Water Park,Women's Store,Yoga Studio
0,"Adyar, Chennai",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000,0.04,0.0,0.0,0.04,0.0
1,Agaram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000,0.00,0.0,0.0,0.00,0.0
2,Alandur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.125,0.00,0.0,0.0,0.00,0.0
3,Alapakkam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000,0.00,0.0,0.0,0.00,0.0
4,Alwarpet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.045455,0.0,0.045455,0.0,0.000,0.00,0.0,0.0,0.00,0.0


## 6 . Finding out Shopping mall in Chennai 

In [55]:
chennai_mall = Chennai_grouped[["Neighbourhood","Shopping Mall"]]

In [56]:
chennai_mall

,Neighbourhood,Shopping Mall
0,"Adyar, Chennai",0.000000
1,Agaram,0.000000
2,Alandur,0.000000
3,Alapakkam,0.000000
4,Alwarpet,0.000000
5,Alwarthirunagar,0.000000
6,Aminjikarai,0.071429
7,Amullaivoyal,0.000000
8,Anna Nagar,0.000000
9,Anna Nagar West,0.000000


In [35]:
len(chennai_mall[chennai_mall["Shopping Mall"] > 0])

7

##  7. Clustering the  Neighborhoods 

In [38]:
# set number of clusters
kclusters = 3

Ch_clustering = chennai_mall.drop(["Neighbourhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Ch_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [39]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ch_merged = chennai_mall.copy()

# add clustering labels
ch_merged["Cluster Labels"] = kmeans.labels_

In [40]:
ch_merged.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)
ch_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"Adyar, Chennai",0.0,0
1,Agaram,0.0,0
2,Alandur,0.0,0
3,Alapakkam,0.0,0
4,Alwarpet,0.0,0


In [43]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ch_merged = ch_merged.join(df_chennai.set_index("Neighbourhood"), on="Neighborhood")

print(ch_merged.shape)
ch_merged.head() # check the last columns!

(156, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Adyar, Chennai",0.0,0,13.00305,80.25193
1,Agaram,0.0,0,13.10953,80.23236
2,Alandur,0.0,0,13.00013,80.20060
3,Alapakkam,0.0,0,13.04610,80.16499
4,Alwarpet,0.0,0,13.03471,80.25416


In [44]:
print(ch_merged.shape)
ch_merged.sort_values(["Cluster Labels"], inplace=True)
ch_merged

(156, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Adyar, Chennai",0.000000,0,13.003050,80.251930
97,Parameshwari Nagar,0.000000,0,13.001920,80.258250
98,"Park Town, Chennai",0.000000,0,13.078670,80.279560
99,Parry's Corner,0.000000,0,13.088678,80.290398
100,"Pattalam, Chennai",0.000000,0,13.097000,80.259220
101,Peddanaickenpettah,0.000000,0,13.098440,80.280100
102,Perambur,0.000000,0,13.122470,80.235690
103,Peravallur,0.000000,0,13.124360,80.228620
104,Periamet,0.000000,0,13.087110,80.269750
105,Periyakudal,0.000000,0,13.081940,80.212780


## 8. Plotting the clusters  with Map 

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ch_merged['Latitude'], ch_merged['Longitude'], ch_merged['Neighborhood'], ch_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 9. Examine each clusters 

In [48]:
ch_merged.loc[ch_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Adyar, Chennai",0.0,0,13.003050,80.251930
97,Parameshwari Nagar,0.0,0,13.001920,80.258250
98,"Park Town, Chennai",0.0,0,13.078670,80.279560
99,Parry's Corner,0.0,0,13.088678,80.290398
100,"Pattalam, Chennai",0.0,0,13.097000,80.259220
101,Peddanaickenpettah,0.0,0,13.098440,80.280100
102,Perambur,0.0,0,13.122470,80.235690
103,Peravallur,0.0,0,13.124360,80.228620
104,Periamet,0.0,0,13.087110,80.269750
105,Periyakudal,0.0,0,13.081940,80.212780


In [49]:
ch_merged.loc[ch_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
20,Chinnakudal,0.071429,1,13.07444,80.22167
140,"Thirumangalam, Chennai",0.100000,1,13.08285,80.19699
26,Chromepet,0.071429,1,12.95234,80.14411
6,Aminjikarai,0.071429,1,13.07139,80.22256


In [50]:
ch_merged.loc[ch_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
151,Vadapalani,0.035714,2,13.05226,80.21120
135,T. Nagar,0.033333,2,13.04536,80.23390
38,"Gopalapuram, Chennai",0.020000,2,13.04613,80.25736
